<a href="https://colab.research.google.com/github/IsaiGowthami/CodeClause_market_basket_analysis_in_python_using_apriori_algorithm/blob/main/market_basket_analysis_in_python_using_apriori_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/IsaiGowthami/CodeClause_market_basket_analysis_in_python_using_apriori_algorithm.git
%cd CodeClause_market_basket_analysis_in_python_using_apriori_algorithm

Cloning into 'CodeClause_market_basket_analysis_in_python_using_apriori_algorithm'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), 21.82 MiB | 12.54 MiB/s, done.
/content/CodeClause_market_basket_analysis_in_python_using_apriori_algorithm/CodeClause_market_basket_analysis_in_python_using_apriori_algorithm/CodeClause_market_basket_analysis_in_python_using_apriori_algorithm


Importing libraries

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

Reading Data & Checking data

In [ ]:
df = pd.read_csv('/content/Online_Retail.csv',encoding='latin1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


Length of data with respect to rows and colomns

In [ ]:
df.shape

(123654, 8)

Checking the data types

In [ ]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

Describing the data

In [ ]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,123654.000000,123653.000000,83147.000000
mean,8.884848,5.225959,15309.075084
std,303.739986,109.762485,1743.352127
min,-74215.000000,0.000000,12346.000000
25%,1.000000,1.250000,13880.000000
50%,3.000000,2.460000,15235.000000
75%,9.000000,4.240000,16907.000000
max,74215.000000,16888.020000,18283.000000


**Data Cleaning**


*   First, some of the descriptions have spaces that need to be removed.
*   We’ll also drop the rows that don’t have invoice numbers.


*   Remove the credit transactions (those with invoice numbers containing C).







In [ ]:
df['Description'] = df['Description'].str.strip() #remove empty spaces
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True) #drop rows that dont have invoice numbers. 
df['InvoiceNo'] = df['InvoiceNo'].astype('str') #convert Invoice No to String
df = df[~df['InvoiceNo'].str.contains('C')] #drop rows with invoice containg C which means Credit Transcation.

Checking the Length of data with respect to rows and colomns after cleaning data

In [ ]:
df.shape

(121400, 8)

After the cleanup, we need to consolidate the items into 1 transaction per row with each product 1 hot encoded. For the sake of keeping the data set small, I’m only looking at sales for France.

In [ ]:
basket = (df[df['Country'] =="France"]        #get data for samples which have Country as France
          .groupby(['InvoiceNo', 'Description'])['Quantity'] #group them on InvoiceNo and Description based on Quantity
          .sum().unstack().reset_index().fillna(0) #sum the quantity, unstack them, fill 0 to nan values. 
          .set_index('InvoiceNo')) #set the index as Invoice Number
      
basket.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,20 DOLLY PEGS RETROSPOT,3 HOOK HANGER MAGIC GARDEN,...,WOODLAND PARTY BAG + STICKER SET,WORLD WAR 2 GLIDERS ASSTD DESIGNS,WRAP ALPHABET DESIGN,WRAP DOILEY DESIGN,WRAP GREEN PEARS,WRAP I LOVE LONDON,WRAP POPPIES DESIGN,WRAP RED APPLES,WRAP VINTAGE PETALS DESIGN,ZINC METAL HEART DECORATION
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Length of basket data with respect to rows and colomns

In [ ]:
basket.shape

(82, 641)

There are a lot of zeros in the data but we also need to make sure any positive values are converted to a 1 and anything less the 0 is set to 0.

In [ ]:
#below function converts a values < 0 to 0 and value greater than equal 1 to 1.  
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
#apply function to data using applymap.
basket_sets = basket.applymap(encode_units)
basket_sets.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,20 DOLLY PEGS RETROSPOT,3 HOOK HANGER MAGIC GARDEN,...,WOODLAND PARTY BAG + STICKER SET,WORLD WAR 2 GLIDERS ASSTD DESIGNS,WRAP ALPHABET DESIGN,WRAP DOILEY DESIGN,WRAP GREEN PEARS,WRAP I LOVE LONDON,WRAP POPPIES DESIGN,WRAP RED APPLES,WRAP VINTAGE PETALS DESIGN,ZINC METAL HEART DECORATION
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now that the data is structured properly, we can generate frequent item sets that have a support of at least 7% (this number was chosen so that I could get enough useful examples)

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

The final step is to generate the rules with their corresponding support, confidence and lift.

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(POSTAGE),(4 TRADITIONAL SPINNING TOPS),0.841463,0.121951,0.109756,0.130435,1.069565,0.007139,1.009756
1,(4 TRADITIONAL SPINNING TOPS),(POSTAGE),0.121951,0.841463,0.109756,0.900000,1.069565,0.007139,1.585366
2,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.073171,0.073171,0.073171,1.000000,13.666667,0.067817,inf
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.073171,0.073171,0.073171,1.000000,13.666667,0.067817,inf
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.073171,0.073171,0.073171,1.000000,13.666667,0.067817,inf


We filter rules to check lift and confidence. This part of the analysis is where the domain knowledge will come in handy. Since we do not have that, we'll just look for a couple of illustrative examples.We can filter the dataframe using standard pandas code. In this case, look for a large lift (6) and high confidence (.8)

In [ ]:
rules[(rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.073171,0.073171,0.073171,1.000000,13.666667,0.067817,inf
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.073171,0.073171,0.073171,1.000000,13.666667,0.067817,inf
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.073171,0.073171,0.073171,1.000000,13.666667,0.067817,inf
5,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.073171,0.073171,0.073171,1.000000,13.666667,0.067817,inf
10,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.073171,0.073171,0.073171,1.000000,13.666667,0.067817,inf
11,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.073171,0.073171,0.073171,1.000000,13.666667,0.067817,inf
208,(WOODLAND PARTY BAG + STICKER SET),(RETROSPOT PARTY BAG + STICKER SET),0.073171,0.097561,0.073171,1.000000,10.250000,0.066032,inf
211,(TABLECLOTH RED APPLES DESIGN),(SET OF 2 TEA TOWELS APPLE AND PEARS),0.085366,0.121951,0.073171,0.857143,7.028571,0.062760,6.146341
220,"(ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...",(ALARM CLOCK BAKELIKE GREEN),0.073171,0.073171,0.073171,1.000000,13.666667,0.067817,inf
221,"(ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...",(ALARM CLOCK BAKELIKE PINK),0.073171,0.073171,0.073171,1.000000,13.666667,0.067817,inf


**Conclusion**

In looking at the rules, it seems that the green and red alarm clocks are purchased together and the red paper cups, napkins and plates are purchased together in a manner that is higher than the overall probability would suggest.

